# Exempel notebook

* Läsa in data från SAS dataset
* Hur man skapar en ny variabel baserat på befintlig variabel
* Hur man konverterar ett string objekt till Pandas datetime variabel
* Några enkla metoder för att förstå sitt data
* Gruppering av data
* Filtrering - välja rader på kriterium
* Flödeslogik och villkor i en funktion applicerad mot dataframe
* Joina dataframes
* Konkatinera dataframes
* Visualisering - Matplotlib
* Objekthantering när dataframes skapas, vy eller fysisikt data - SettingWithWarningCopy
* Plocka rader och kolumner i dataframe med iloc metoden


__En mycket bra resurs att söka hjälp i är pandas egen dokumentation: https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sas.html__

__I Table of Contents finns "Comparison with SAS". I denna så jämför man pandas med SAS utifrån flera olika perspektiv. Jag brukar alltid ha den öppen när jag arbetar med analys i pandas__



In [1]:
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt

In [2]:
?pd.read_sas

In [3]:
type(pd.read_sas)

function

In [4]:
type(plt)

module

In [5]:
type(pd)

module

## 1. Läsa in data från SAS format

In [6]:
df_staff = pd.read_sas('staff.sas7bdat', encoding = 'Latin8')

In [7]:
# Information om det inlästa datat
df_staff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Employee_ID    424 non-null    float64       
 1   Job_Title      424 non-null    object        
 2   Salary         424 non-null    float64       
 3   Gender         424 non-null    object        
 4   Birth_Date     424 non-null    datetime64[ns]
 5   Emp_Hire_Date  424 non-null    datetime64[ns]
 6   Emp_Term_Date  116 non-null    datetime64[ns]
 7   Manager_ID     423 non-null    float64       
 8   start_date     424 non-null    object        
 9   end_date       424 non-null    object        
dtypes: datetime64[ns](3), float64(3), object(4)
memory usage: 33.2+ KB


In [8]:
# Tittar på de 5 första observationerna
df_staff.head()

,Employee_ID,Job_Title,Salary,Gender,Birth_Date,Emp_Hire_Date,Emp_Term_Date,Manager_ID,start_date,end_date
0,120101.0,Director,163040.0,M,1980-08-18,2007-07-01,NaT,120261.0,2007-07-01,9999-12-31
1,120102.0,Sales Manager,108255.0,M,1973-08-11,1993-06-01,NaT,120101.0,1993-06-01,9999-12-31
2,120103.0,Sales Manager,87975.0,M,1953-01-22,1978-01-01,NaT,120101.0,1978-01-01,9999-12-31
3,120104.0,Administration Manager,46230.0,F,1958-05-11,1985-01-01,NaT,120101.0,1985-01-01,9999-12-31
4,120105.0,Secretary I,27110.0,F,1978-12-21,2003-05-01,NaT,120101.0,2003-05-01,9999-12-31


### Dataframen innehåller två datatyper: float64 samt object
* float64 - numerisk variabel med decimaler
* object - pandas datatyp för character
* datetime64 - pandas datatyp för datum

## 2. Hur skapar man en ny variabel i Pandas

In [9]:
df_staff['Double'] = df_staff['Salary'] * 2

In [10]:
df_staff.head()

,Employee_ID,Job_Title,Salary,Gender,Birth_Date,Emp_Hire_Date,Emp_Term_Date,Manager_ID,start_date,end_date,Double
0,120101.0,Director,163040.0,M,1980-08-18,2007-07-01,NaT,120261.0,2007-07-01,9999-12-31,326080.0
1,120102.0,Sales Manager,108255.0,M,1973-08-11,1993-06-01,NaT,120101.0,1993-06-01,9999-12-31,216510.0
2,120103.0,Sales Manager,87975.0,M,1953-01-22,1978-01-01,NaT,120101.0,1978-01-01,9999-12-31,175950.0
3,120104.0,Administration Manager,46230.0,F,1958-05-11,1985-01-01,NaT,120101.0,1985-01-01,9999-12-31,92460.0
4,120105.0,Secretary I,27110.0,F,1978-12-21,2003-05-01,NaT,120101.0,2003-05-01,9999-12-31,54220.0


## 3. Konvertera till datetimeformat

### Vi behöver konvertera datumvariablerna (Start_Date samt End:Date) till datetime - datumvariabel i Pandas. Nu ligger den som objekt och då kan vi inte applicera datum metoder på denna. För att göra den konverteringen använder vi pandas funktion to_datetime


In [11]:
?pd.to_datetime

In [12]:
df_staff['P_start_date'] = pd.to_datetime(df_staff['start_date'])

In [13]:
df_staff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Employee_ID    424 non-null    float64       
 1   Job_Title      424 non-null    object        
 2   Salary         424 non-null    float64       
 3   Gender         424 non-null    object        
 4   Birth_Date     424 non-null    datetime64[ns]
 5   Emp_Hire_Date  424 non-null    datetime64[ns]
 6   Emp_Term_Date  116 non-null    datetime64[ns]
 7   Manager_ID     423 non-null    float64       
 8   start_date     424 non-null    object        
 9   end_date       424 non-null    object        
 10  Double         424 non-null    float64       
 11  P_start_date   424 non-null    datetime64[ns]
dtypes: datetime64[ns](4), float64(4), object(4)
memory usage: 39.9+ KB


In [14]:
df_staff.head()

,Employee_ID,Job_Title,Salary,Gender,Birth_Date,Emp_Hire_Date,Emp_Term_Date,Manager_ID,start_date,end_date,Double,P_start_date
0,120101.0,Director,163040.0,M,1980-08-18,2007-07-01,NaT,120261.0,2007-07-01,9999-12-31,326080.0,2007-07-01
1,120102.0,Sales Manager,108255.0,M,1973-08-11,1993-06-01,NaT,120101.0,1993-06-01,9999-12-31,216510.0,1993-06-01
2,120103.0,Sales Manager,87975.0,M,1953-01-22,1978-01-01,NaT,120101.0,1978-01-01,9999-12-31,175950.0,1978-01-01
3,120104.0,Administration Manager,46230.0,F,1958-05-11,1985-01-01,NaT,120101.0,1985-01-01,9999-12-31,92460.0,1985-01-01
4,120105.0,Secretary I,27110.0,F,1978-12-21,2003-05-01,NaT,120101.0,2003-05-01,9999-12-31,54220.0,2003-05-01


### Nu kan vi applicera metoder mot datumvariabeln

In [15]:
type(df_staff['P_start_date'])

pandas.core.series.Series

In [20]:
# För att se vilka attribut som finns tillgängliga på en serie
test = df_staff['P_start_date']

In [22]:
dir(df_staff['P_start_date'])

['T',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__

In [23]:
?test.dt

### Vi skapar år, månad och dag

In [28]:
df_staff['Year'] = df_staff['P_start_date'].dt.year
df_staff['Month'] = df_staff['P_start_date'].dt.month
df_staff['day'] = df_staff['P_start_date'].dt.month


In [ ]:
df_staff.head()

## 4. Exempel på metoder för att enkelt förstå sitt data

In [ ]:
# Fördelning män och kvinnor
df_staff['Gender'].value_counts()

In [ ]:
# Medelvärde lön
df_staff['Salary'].mean()


In [ ]:
# Univariat statistik på alla numeriska variabler i dataframen
df_staff.describe()

## 5. Gruppering av data. 
### För att gruppera sitt data skapar man ett sk grouped objekt. Ett grouped objekt innehåller metadata som beskriver hur grupperingen är gjord. På detta objekt kan man sedan välja kolumner att ta fram olika statistik på. Detta sker - det vet ni nu - självklart med en lämplig metod.

In [24]:
# Vi skapar ett grouped objekt på Gender

grouped = df_staff.groupby('Gender')

In [25]:
type(grouped)

pandas.core.groupby.generic.DataFrameGroupBy

In [26]:
# Finns det någon skillnad i lön mellan män och kvinnor?. Väljer Serien Salery från groupedby objektet och applicerar 
# metoden mean

grouped['Salary'].mean()

Gender
F    35591.308901
M    40050.042918
Name: Salary, dtype: float64

In [ ]:
df_staff.info()

In [29]:
### Man kan självklart gruppera på mer än en variabel. Observera att man då lägger grupperingsvariablerna i en lista

grouped_multi =  df_staff.groupby(['Year','Gender'])

In [ ]:
round(grouped_multi['Salary'].mean())

## 6. Filtrering - välja rader på kriterium

In [30]:
# Vi skapar en ny dataframe som enbart innehåller kvinnor
df_staff_female = df_staff[df_staff['Gender'] == 'F']

In [31]:
df_staff_female.count()

Employee_ID      191
Job_Title        191
Salary           191
Gender           191
Birth_Date       191
Emp_Hire_Date    191
Emp_Term_Date     52
Manager_ID       191
start_date       191
end_date         191
Double           191
P_start_date     191
Year             191
Month            191
day              191
dtype: int64

### Vad är det vi gör här egentligen?

In [32]:
df_staff['Truth'] = df_staff['Gender'] == 'F'

In [33]:
# Väljer ett subset av variabler
df_staff[['Gender','Truth']].head()

,Gender,Truth
0,M,False
1,M,False
2,M,False
3,F,True
4,F,True


In [ ]:
### Subsetting with Calculated Values - exempel SQL

'''
proc sql;
select Gender, Salary, Bonus,
       Salary * .10 as Bonus
   from orion.employee_payroll
   where calculated Bonus < 3000;
quit; 

'''


In [34]:
df_staff['Bonus'] = df_staff['Salary'] * 0.1
df_staff_subset = df_staff[df_staff['Bonus'] > 3000][[ 'Gender', 'Salary','Bonus']]

In [35]:
df_staff_subset.head(10)

,Gender,Salary,Bonus
0,M,163040.0,16304.0
1,M,108255.0,10825.5
2,M,87975.0,8797.5
3,F,46230.0,4623.0
6,F,30475.0,3047.5
13,F,31285.0,3128.5
16,M,31670.0,3167.0
18,M,30255.0,3025.5
24,M,32040.0,3204.0
27,F,30890.0,3089.0


In [36]:
df_staff_subset['Gender'].value_counts()

M    102
F     84
Name: Gender, dtype: int64

## 7. Flödeslogik och villkor i en funktion applicerad mot dataframe
### Förfina logiken med egendiefinerad funktion som ancänds av apply metoden i en lambda funktion. Om det är en kvinna ska bonusen vara 20%, om en man -20%.

In [37]:
def metoo(gender, salery):
    if gender == 'F':
        bonus = salery * 0.2
    else:
        bonus = salery * (-0.2)
    return bonus

In [38]:
bonus = metoo('M',100)
bonus

-20.0

### OBS! Förstår man nedan har man ett mycket kraftfullt verktyg för att manipupelera data på radnivå med Pandas

In [39]:
df_staff['Mee_to_Bonus'] = df_staff.apply(lambda x: metoo(x['Gender'], x['Salary']), axis = 1) 

### Metoden apply appliceras på objektet df_staff. Denna metod tillåter att man skickar in en funktion. I exemplet ovan är det en lambda funktion som använder en egendefineradfunktion som argument (metoo). På varje rad appliceras funktionen metoo med argumenten df_staff['Gender'] och df_staff['Salary]. Lambda funktionen erbjuder möjligheter att använda funktioner i sammanhang där den vanliga funktionen inte fungerar. Det är väl använd tid att sätta sig in i lambda konceptet.

In [40]:
df_staff[['Gender','Salary','Mee_to_Bonus']].head(10)

,Gender,Salary,Mee_to_Bonus
0,M,163040.0,-32608.0
1,M,108255.0,-21651.0
2,M,87975.0,-17595.0
3,F,46230.0,9246.0
4,F,27110.0,5422.0
5,M,26960.0,-5392.0
6,F,30475.0,6095.0
7,F,27660.0,5532.0
8,F,26495.0,5299.0
9,M,28615.0,-5723.0


In [41]:
?df_staff.apply

In [42]:
type(df_staff.apply)

method

## 8. Joina dataframes 

In [ ]:
df_cust = pd.read_sas('customer2.sas7bdat', encoding = 'Latin8')
df_trans = pd.read_sas('transaction2.sas7bdat', encoding = 'Latin8')

In [ ]:
df_cust.info()

In [ ]:
df_trans.info()

In [ ]:
df_cust.head()

In [ ]:
df_cust = df_cust.drop_duplicates('ID')
df_cust.head()

In [ ]:
df_trans.head()

In [ ]:
# Inner join
df_merged_inner = df_cust.merge(df_trans, on = ['ID'], how = 'inner')
df_merged_inner.head()


In [ ]:
# Left join 

df_merged_left = df_cust.merge(df_trans, on = ['ID'], how = 'left')
df_merged_left.head()

In [ ]:
# Outer join

df_merged_outer = df_cust.merge(df_trans, on = ['ID'], how = 'outer')
df_merged_outer.head(6)

## 9. Konkatinera dataframes

In [ ]:
df_merged = df_merged_outer.copy() 

In [ ]:
df_merged

In [ ]:
concat_df = pd.concat([df_merged, df_merged_outer ], axis = 0)

In [ ]:
concat_df.head(20)

## 9. Split - Apply - Combine
### Aggregering - summera data i en serie för att returnera en skalär.  Vi vill ta fram statistik över en gruppering på data: split- apply-combine konceptet

* Split - Datat delas upp i delar på vald grupperingsvariabel
* Apply - På delarna appliceras logik som returnerar skalär
* Combine - Delarna sätts ihop på grupperingsvariabeln

In [43]:
# Läser in nytt exempeldata.

df_order_fact = pd.read_sas('order_fact.sas7bdat', encoding = 'Latin8')
df_product_dim = pd.read_sas('product_dim.sas7bdat', encoding = 'Latin8')

In [44]:
df_product_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Product_ID        481 non-null    float64
 1   Product_Line      481 non-null    object 
 2   Product_Category  481 non-null    object 
 3   Product_Group     481 non-null    object 
 4   Product_Name      481 non-null    object 
 5   Supplier_Country  481 non-null    object 
 6   Supplier_Name     481 non-null    object 
 7   Supplier_ID       481 non-null    float64
dtypes: float64(2), object(6)
memory usage: 30.2+ KB


In [45]:
df_product_dim.head()

,Product_ID,Product_Line,Product_Category,Product_Group,Product_Name,Supplier_Country,Supplier_Name,Supplier_ID
0,2.102001e+11,Children,Children Sports,"A-Team, Kids","Kids Sweat Round Neck,Large Logo",US,A Team Sports,3298.0
1,2.102001e+11,Children,Children Sports,"A-Team, Kids",Sweatshirt Children's O-Neck,US,A Team Sports,3298.0
2,2.102002e+11,Children,Children Sports,"Bathing Suits, Kids",Sunfit Slow Swimming Trunks,US,Nautlius SportsWear Inc,6153.0
3,2.102002e+11,Children,Children Sports,"Bathing Suits, Kids",Sunfit Stockton Swimming Trunks Jr.,US,Nautlius SportsWear Inc,6153.0
4,2.102003e+11,Children,Children Sports,"Eclipse, Kid's Clothes",Fleece Cuff Pant Kid'S,US,Eclipse Inc,1303.0


In [46]:
df_order_fact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Customer_ID         617 non-null    float64       
 1   Employee_ID         617 non-null    float64       
 2   Street_ID           617 non-null    float64       
 3   Order_Date          617 non-null    datetime64[ns]
 4   Delivery_Date       617 non-null    datetime64[ns]
 5   Order_ID            617 non-null    float64       
 6   Order_Type          617 non-null    float64       
 7   Product_ID          617 non-null    float64       
 8   Quantity            617 non-null    float64       
 9   Total_Retail_Price  617 non-null    float64       
 10  CostPrice_Per_Unit  617 non-null    float64       
 11  Discount            3 non-null      float64       
dtypes: datetime64[ns](2), float64(10)
memory usage: 58.0 KB


In [47]:
### Joinar fakta tabell mot dimensonstabellen och lägger på produktkategori

df_analys = df_order_fact.merge(df_product_dim[['Product_ID','Product_Line','Product_Category']],
                                on = ['Product_ID'], how = 'inner')

In [48]:
df_analys.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 617 entries, 0 to 616
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Customer_ID         617 non-null    float64       
 1   Employee_ID         617 non-null    float64       
 2   Street_ID           617 non-null    float64       
 3   Order_Date          617 non-null    datetime64[ns]
 4   Delivery_Date       617 non-null    datetime64[ns]
 5   Order_ID            617 non-null    float64       
 6   Order_Type          617 non-null    float64       
 7   Product_ID          617 non-null    float64       
 8   Quantity            617 non-null    float64       
 9   Total_Retail_Price  617 non-null    float64       
 10  CostPrice_Per_Unit  617 non-null    float64       
 11  Discount            3 non-null      float64       
 12  Product_Line        617 non-null    object        
 13  Product_Category    617 non-null    object        

In [49]:
df_analys['Product_Category'].value_counts()

Outdoors                    147
Clothes                     121
Assorted Sports Articles     79
Golf                         74
Shoes                        38
Team Sports                  35
Running - Jogging            30
Children Sports              29
Racket Sports                23
Winter Sports                17
Indoor Sports                16
Swim Sports                   8
Name: Product_Category, dtype: int64

In [50]:
df_analys['Product_Line'].value_counts()

Sports             282
Clothes & Shoes    159
Outdoors           147
Children            29
Name: Product_Line, dtype: int64

In [51]:
# Andel av totalen
round(df_analys['Product_Line'].value_counts()/len(df_analys),2)

Sports             0.46
Clothes & Shoes    0.26
Outdoors           0.24
Children           0.05
Name: Product_Line, dtype: float64

### Nu vill vi ta fram statistik på grupperingen Product Line - vi använder split - apply - combine konceptet. Först med Pandas optimerade metoder. Dessa anropas med agg metoden - returnerar skalär 

In [52]:
grouped = df_analys.groupby(['Product_Line','Product_Category']) 

In [53]:
result = grouped['Total_Retail_Price'].agg('mean')
result

Product_Line     Product_Category        
Children         Children Sports              55.737931
Clothes & Shoes  Clothes                      91.269339
                 Shoes                       150.028947
Outdoors         Outdoors                    203.142857
Sports           Assorted Sports Articles    185.907595
                 Golf                        233.597027
                 Indoor Sports               398.950000
                 Racket Sports               144.969565
                 Running - Jogging           112.926667
                 Swim Sports                  47.887500
                 Team Sports                  58.393429
                 Winter Sports               255.842647
Name: Total_Retail_Price, dtype: float64

In [54]:
### Du kan använda flera funktioner i samma anrop
funtions = ['count','mean','max']
result1 = grouped['Total_Retail_Price'].agg(funtions)
result1



count        mean     max
Product_Line    Product_Category                                   
Children        Children Sports              29   55.737931   134.0
Clothes & Shoes Clothes                     121   91.269339   408.8
                Shoes                        38  150.028947   406.0
Outdoors        Outdoors                    147  203.142857  1687.5
Sports          Assorted Sports Articles     79  185.907595  1064.0
                Golf                         74  233.597027  1937.2
                Indoor Sports                16  398.950000  1561.8
                Racket Sports                23  144.969565  1250.4
                Running - Jogging            30  112.926667   403.5
                Swim Sports                   8   47.887500    70.2
                Team Sports                  35   58.393429   155.6
                Winter Sports                17  255.842647   949.8

### Om vi inte är nöjda med de defaulta namnen -använd en tuple

In [55]:
### Du kan använda flera funktioner i samma anrop
funtions2 = [('N','count'),('Medel','mean'),('Max','max')]

result2 = grouped['Total_Retail_Price'].agg(funtions2)
result2

N       Medel     Max
Product_Line    Product_Category                                 
Children        Children Sports            29   55.737931   134.0
Clothes & Shoes Clothes                   121   91.269339   408.8
                Shoes                      38  150.028947   406.0
Outdoors        Outdoors                  147  203.142857  1687.5
Sports          Assorted Sports Articles   79  185.907595  1064.0
                Golf                       74  233.597027  1937.2
                Indoor Sports              16  398.950000  1561.8
                Racket Sports              23  144.969565  1250.4
                Running - Jogging          30  112.926667   403.5
                Swim Sports                 8   47.887500    70.2
                Team Sports                35   58.393429   155.6
                Winter Sports              17  255.842647   949.8

### Slutligen gör vi en egen funktion som vi använder apply mot

### Vi vill plocka de fem högs betalda anställda per kön i df_staff

In [ ]:
df_staff.info()

In [56]:
# Gör din egna funktion
def top(df, n = 5, column ='Salary'): 
    return df.sort_values(by = column)[-n:]

In [57]:
top(df_staff)

,Employee_ID,Job_Title,Salary,Gender,Birth_Date,Emp_Hire_Date,Emp_Term_Date,Manager_ID,start_date,end_date,Double,P_start_date,Year,Month,day,Truth,Bonus,Mee_to_Bonus
416,121141.0,Vice President,194885.0,M,1948-06-19,1978-01-01,NaT,120261.0,1978-01-01,9999-12-31,389770.0,1978-01-01,1978,1,1,False,19488.5,-38977.0
99,120260.0,Chief Marketing Officer,207885.0,F,1968-12-02,1988-11-01,NaT,120259.0,1988-11-01,9999-12-31,415770.0,1988-11-01,1988,11,11,True,20788.5,41577.0
100,120261.0,Chief Sales Officer,243190.0,M,1973-02-21,1991-08-01,NaT,120259.0,1991-08-01,9999-12-31,486380.0,1991-08-01,1991,8,8,False,24319.0,-48638.0
101,120262.0,Chief Financial Officer,268455.0,M,1973-10-21,1992-09-01,NaT,120259.0,1992-09-01,9999-12-31,536910.0,1992-09-01,1992,9,9,False,26845.5,-53691.0
98,120259.0,Chief Executive Officer,433800.0,M,1968-01-25,1993-09-01,NaT,NaN,1993-09-01,9999-12-31,867600.0,1993-09-01,1993,9,9,False,43380.0,-86760.0


In [58]:
# Nu grupperar vi på kön och plockar de med högsta lönerna per grupp

df_staff.groupby('Gender').apply(top)

Employee_ID                 Job_Title    Salary Gender Birth_Date  \
Gender                                                                          
F      248     120798.0    Senior Project Manager   80755.0      F 1963-06-23   
       419     121144.0             Sales Manager   83505.0      F 1968-06-28   
       125     120661.0  Senior Logistics Manager   85495.0      F 1958-11-27   
       169     120719.0  Senior Marketing Manager   87420.0      F 1973-01-22   
       99      120260.0   Chief Marketing Officer  207885.0      F 1968-12-02   
M      0       120101.0                  Director  163040.0      M 1980-08-18   
       416     121141.0            Vice President  194885.0      M 1948-06-19   
       100     120261.0       Chief Sales Officer  243190.0      M 1973-02-21   
       101     120262.0   Chief Financial Officer  268455.0      M 1973-10-21   
       98      120259.0   Chief Executive Officer  433800.0      M 1968-01-25   

           Emp_Hire_Date Emp_Term_Date  Manager_ID  start_date    end_date  \
Gender                                                                       
F      248    1991-01-01           NaT    120800.0  1991-01-01  9999-12-31   
       419    1995-11-01           NaT    121142.0  1995-11-01  9999-12-31   
       125    1988-01-01    2007-06-30    120659.0  1988-01-01  2007-06-30   
       169    2000-02-01           NaT    120260.0  2000-02-01  9999-12-31   
       99     1988-11-01           NaT    120259.0  1988-11-01  9999-12-31   
M      0      2007-07-01           NaT    120261.0  2007-07-01  9999-12-31   
       416    1978-01-01           NaT    120261.0  1978-01-01  9999-12-31   
       100    1991-08-01           NaT    120259.0  1991-08-01  9999-12-31   
       101    1992-09-01           NaT    120259.0  1992-09-01  9999-12-31   
       98     1993-09-01           NaT         NaN  1993-09-01  9999-12-31   

              Double P_start_date  Year  Month  day  Truth    Bonus  \
Gender                                                                
F      248  161510.0   1991-01-01  1991      1    1   True   8075.5   
       419  167010.0   1995-11-01  1995     11   11   True   8350.5   
       125  170990.0   1988-01-01  1988      1    1   True   8549.5   
       169  174840.0   2000-02-01  2000      2    2   True   8742.0   
       99   415770.0   1988-11-01  1988     11   11   True  20788.5   
M      0    326080.0   2007-07-01  2007      7    7  False  16304.0   
       416  389770.0   1978-01-01  1978      1    1  False  19488.5   
       100  486380.0   1991-08-01  1991      8    8  False  24319.0   
       101  536910.0   1992-09-01  1992      9    9  False  26845.5   
       98   867600.0   1993-09-01  1993      9    9  False  43380.0   

            Mee_to_Bonus  
Gender                    
F      248       16151.0  
       419       16701.0  
       125       17099.0  
       169       17484.0  
       99        41577.0  
M      0        -32608.0  
       416      -38977.0  
       100      -48638.0  
       101      -53691.0  
       98       -86760.0

## 10. Visualisering - Matplotlib
### Det finns ett helt ekosystem för visaulisering. Grundmodulen för detta är matplotlib som redan är importerad i denna notebook. Vi ska göra två enkla visualiseringar för att visa konceptet

In [ ]:
# Definierar hela arean
fig = plt.figure(figsize = (10,5))
# Skapar grafobjekt som ska läggas in i arean ovan
ax = fig.add_subplot(1,1,1)

ax.set_title('Inkomst fördelat på kön')

bar_serie = round(df_staff.groupby('Gender')['Salary'].mean())

bar_serie.plot(kind='bar', rot = 0, grid = False, alpha = 0.6)

plt.show()

### Vad ska man tänka på här? Det finns oändligt många parametrar att sätta?
### Alla dataframes och serier har en plot metod. 
### Om man sätter ett index blir detta index alltid X axeln
### Det enda sättet är att pröva sig fram med olika grafer, se hjälpen nedan
### Bra att känna till är att man kan annotera samt skapa flera grafer i samma area


In [ ]:
?bar_serie.plot

### Exenmpel annotering samt två grafer i samma bild

In [ ]:
fig = plt.figure(figsize = (18,9))
# Skapar grafobjekt som ska läggas in i arean ovan
# Graf1
ax1 = fig.add_subplot(2,1,1)
# Graf2
ax2 = fig.add_subplot(2,1,2)

ax1.set_title('Inkomst fördelat på kön')

ax2.set_title('Antal anställda per år')

bar_serie = round(df_staff.groupby('Gender')['Salary'].mean())

line_serie = round(df_staff.groupby('Year')['Gender'].count())

bar_serie.plot(kind='bar', ax = ax1, rot = 0, grid = False, alpha = 0.6)

line_serie.plot(kind='line', ax = ax2, rot = 0, style = 'k-')

### Nedan logik för annotering i graf2

ax2.annotate('Vad händer 2010?', 
            xy = (2010, 95),
            xytext = (2000, 80),
            arrowprops = dict(facecolor = 'black', shrink = 0.1, width = 2),
            horizontalalignment = 'left')

plt.show()

In [ ]:
?ax2.annotate

In [ ]:
bar_serie

In [ ]:
type(bar_serie)

In [ ]:
line_serie

### Objekthantering när du skapar en ny dataframe, vy eller fysiskt data
__Det sätt som Pandas skapar en ny dataframe, vy eller nytt fysiskt data kan ge varningen "SettingWithCopyWarning". Jag kommer inte att ge me in på djupet på detta men nedan ett exempel. När denna varning uppstår så försöker ni göra något mot en dataframe via en vy som inte slår igenom i det underliggande datat. Och man ska också vara medveten om att i vissa fall så arbetar men mot en vy som faktiskt slår igenom i det underliggande datat__

### Först manipulation av vy som påverkar ett underliggande objekt

In [ ]:
df_order_fact = pd.read_sas('order_fact.sas7bdat', encoding = 'Latin8')

In [ ]:
id(df_order_fact)

In [ ]:
df_new = df_order_fact

In [ ]:
id(df_new)

In [ ]:
df_order_fact.info()

In [ ]:
df_new['Q2'] = df_new.Quantity * df_new.Quantity

In [ ]:
df_new.info()

In [ ]:
df_order_fact.info()

### Vill man vara helt säker på vad man gör använd copy metod för att skapa ett nytt objekt

In [ ]:
df_order_fact2 = pd.read_sas('order_fact.sas7bdat', encoding = 'Latin8')

In [ ]:
id(df_order_fact2)

In [ ]:
df_new2 = df_order_fact2.copy()

In [ ]:
id(df_new2)

In [ ]:
df_new2['Q2'] = df_new2.Quantity * df_new2.Quantity 

In [ ]:
df_new2.info()

In [ ]:
df_order_fact2.info()

__Nu provocerar vi fram en SettingWithWarningCopy__

In [ ]:
df_new

__Låt oss anta att för alla rader med Quantity = 1 är fel, de ska vara 10 istället. Vi måste uppdarera dataframen__

In [ ]:
df_new[df_new.Quantity == 1]['Q2'] = 100

In [ ]:
df_new[df_new.Quantity == 1][['Quantity','Q2']]

### Försök till förklaring

__Metoden df_new[df_new.Quantity == 1] skapar en vy av den ursprunliga dataframen (get method) , sedan gör vi en assignment ['Q2'] = 100 (assigmnent method) som inte slår igenom mot det underliggande objektet, det fysiska data. Lösningen är att arbete med loc metoden som resulterar i en operation och som säkerställer att det underliggande datat blir uppdaterat__ 

### Slicing loc metoden
* Välj rad och kolumn genom att ange index och variabel

In [ ]:
df_new.loc[df_new.Quantity == 1,'Q2'] = 100

In [ ]:
df_new[df_new.Quantity == 1][['Quantity','Q2']]

In [ ]:
id(df_new)

In [ ]:
id(df_order_fact)

In [ ]:
df_order_fact[df_order_fact.Quantity == 1][['Quantity','Q2']]

# Take away!

__Aldrig ignorerar denna varning!.__

# Slicing in a dataframe: Skär ut rader och kolumner genom positioner (index) - iloc

In [ ]:
# df_new är en vy mot df_order_fact
df_new

In [ ]:
df_new.iloc?

In [ ]:
## Välja kolumner utan metod genom att peka på kolumner
df_new[['Customer_ID','Employee_ID']]

In [ ]:
# Select row and columns with iloc method
df_new.iloc[0,0]

In [ ]:
df_new.iloc[1,0]

In [ ]:
df_new.iloc[:2,:2]

In [ ]:
df_new.iloc[-4:,:2]

In [ ]:
df_order_fact2.loc?